In [28]:
import json,os
import numpy as np
import pandas as pd

### Cargo datos

In [29]:
ligue2team2rank = json.load( open("../../data/ligue2team2rank.json", "r") )

In [30]:
matches = pd.read_csv("../../data/matches.csv")
print(matches.shape)
matches.head(2)

(1826, 8)


,match,home,away,winner,score_home,score_away,label,liga
0,2500089,1646,1659,1659,1,2,"Burnley - AFC Bournemouth, 1 - 2",England
1,2500090,1628,1627,1628,2,0,"Crystal Palace - West Bromwich Albion, 2 - 0",England


In [ ]:
metrics = pd.read_csv("../../data/metrics_2.csv")
print(metrics.shape)
metrics.head(2)

(3652, 13)


,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga
0,2499719,1609,0.852098,-0.228674,-0.391992,1.375792,1.008434,2.412985,0.260042,0.561725,-0.711431,3.181387,England
1,2499719,1631,-1.508043,-1.393953,-1.280744,1.260816,-1.129144,-0.594481,-0.966294,-0.996953,-0.990073,-1.372080,England


### Armo redes

Primero cruzo los datos por match

In [32]:
df0 = pd.merge(metrics, matches[['match', 'home', 'away', 'winner', 'score_home','score_away']], on='match', how='left')

print(df0.shape)
df0.head(2)

(3652, 18)


,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga,home,away,winner,score_home,score_away
0,2499719,1609,0.852098,-0.228674,-0.391992,1.375792,1.008434,2.412985,0.260042,0.561725,-0.711431,3.181387,England,1609,1631,1609,4,3
1,2499719,1631,-1.508043,-1.393953,-1.280744,1.260816,-1.129144,-0.594481,-0.966294,-0.996953,-0.990073,-1.372080,England,1609,1631,1609,4,3


In [33]:
metrics[metrics["liga"]=="England"]['team1'].unique()

array([ 1609,  1631,  1625,  1651,  1610,  1646,  1628,  1673,  1623,
        1639,  1611,  1633,  1613,  1624,  1619, 10531,  1612,  1644,
        1627,  1659])

In [34]:
metricas = ['direct_play','counterattack','T_maintenance','T_build_up',
            'T_zona_media','flow_rate','crossing','pressure_point','pressure_loss',	
            'shots']

ligas = ['England','France','Germany','Italy','Spain']


for liga in ligas:

    carpeta = f'{liga}'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)


    team2rank = ligue2team2rank[liga]
    df = df0[ df0['liga']==liga ]


    for metrica in metricas:

        d={}

        def f(row):

            # tomo datos
            team1 = str(row.team1)
            nodo1 = team2rank[team1]

            if row.team1==row.home:
                team2=str(row.away)
            else:
                team2=str(row.home)

            nodo2 = team2rank[team2]

            w = row[metrica] 

            try:
                d[(nodo1,nodo2)]+=w
            except:
                d[(nodo1,nodo2)]=w
            

        df.agg( lambda x: f(x), axis=1 )


        # armo red
        Nmax = max(elemento for tupla in d for elemento in tupla)
        red = []

        for i in range(1,Nmax):

            for j in range(i+1, Nmax+1):

                fij = d[(j,i)] - d[(i,j)] 

                red.append( [i,j, fij] )



        #guardo data
        arx_out = f'{liga}/red_{metrica}.dat'

        np.savetxt(
            arx_out,               
            red,               
            fmt='%d,%d,%.4e',         
            delimiter=',', 
            header='n1,n2,fij',
            comments=''
        )

